# 文生图-Stable Diffusion

同济子豪兄 2023-6-13

https://github.com/open-mmlab/mmagic/tree/main/configs/stable_diffusion

## 导入工具包

In [ ]:
from mmagic.apis import MMagicInferencer

## 载入Stable Diffusion模型

In [ ]:
sd_inferencer = MMagicInferencer(model_name='stable_diffusion')

## 指定Prompt文本

In [ ]:
text_prompt1 = 'A panda is having dinner at KFC'

text_prompt2 = 'A Persian cat walking in the streets of New York'

## 执行预测

In [ ]:
image = sd_inferencer.infer(text=text_prompt1, result_out_dir='output/sd_res.png')[1]
image

In [ ]:
image = sd_inferencer.infer(text=text_prompt2, result_out_dir='output/sd_res.png')[1]
image